Клевцов Тимур, М05-013д   
Визуализация дерева алгоритмом Layered-Tree-Draw c сжатием

In [163]:
import math
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import networkx as nx
from itertools import *

Прописать имя файла с графом в name. Еще в кавычках есть граф с лекции

In [164]:
name = "tree-84n.xml"
G = nx.read_graphml("./%s" % name)
graph = []
infty = 10000
mark_v = 0  #глобальный счетчик для маркировки

'''
G = nx.DiGraph()
G.add_nodes_from([i for i in range(11)])
edges = [
  (0,6), (0,1),
  (1,3), (1,2),
  (3,4), (3,5),
  (6,10),
  (10,7),
  (7,9), (7,8)
]
G.add_edges_from(edges)
'''

class my_node():
  label = infty  #означает непомеченную вершину
  hight = 0 #расстояние до корня
  x = 0
  y = 0
  childs = []
  parents = []
  def __init__(self, label, h, x, y, childs, parents):
    self.label = label
    self.x = x
    self.y = y
    self.childs = childs
    self.parents = parents
  def __repr__(self):
    s = "({}, hight={}, position=({},{}), parents={}, childs={})".format(
        self.label, self.hight, self.x, self.y, self.parents, self.childs)
    return s 

def create_graph():
  nodes_dict = {}
  number = 0
  for node in G.nodes():
    nodes_dict[node] = number
    graph.append(my_node(infty, 0, 0, 0, [], []))
    number += 1
  for edge in G.edges():
    v0 = nodes_dict[edge[0]]
    v1 = nodes_dict[edge[1]]
    graph[v0].childs.append(v1)
    graph[v1].parents.append(v0)
  return

def order_reversal(v, h):
  global mark_v
  if len(graph[v].childs) == 0:
    graph[v].label = mark_v
    graph[v].hight = h
    mark_v += 1
    return
  order_reversal(graph[v].childs[0], h + 1)
  graph[v].label = mark_v
  graph[v].hight = h
  mark_v += 1
  if len(graph[v].childs) == 2:
    order_reversal(graph[v].childs[1], h + 1)
  return

def distance(left_side, right_side):
  l = min(len(left_side), len(right_side))
  internal = left_side[:l] - right_side[:l] + 1
  return internal.max()

def center_counting(left, right):
  if left == right:
    return 1
  c = (left - right) / 2
  return int(np.sign(c) * math.ceil(abs(c)))

def archiving(v):  #рекурсивное сжатие дерева по Х
  if len(graph[v].childs) == 0:
    #print(v, "[0] [0]")
    return np.array([0]), np.array([0])
  if len(graph[v].childs) == 1:
    left, right = archiving(graph[v].childs[0])
    right_L = max(right.max(), 0)
    left_L = abs(min(left.min(), 0))
    shift = center_counting(left_L, right_L)
    graph[graph[v].childs[0]].x = shift
    left, right = left + shift, right + shift
    left = np.concatenate([[0], left])
    right = np.concatenate([[0], right])
    #print(v, left, right)
    return left, right
  if len(graph[v].childs) == 2:
    left_left, left_right = archiving(graph[v].childs[0]) 
    right_left, right_right = archiving(graph[v].childs[1])
    L = distance(left_right, right_left)
    if L == 1:
      L = 2
    shift_right, shift_left = L // 2, - L + L // 2
    graph[graph[v].childs[0]].x = shift_left
    graph[graph[v].childs[1]].x = shift_right
    left_left += shift_left
    left_right += shift_left
    right_right += shift_right
    right_left += shift_right
    l = min(len(right_right), len(left_left))
    left = np.concatenate([[0], left_left[:], right_left[l:]])
    right = np.concatenate([[0], right_right[:], left_right[l:]])
    #print(v, left_left, left_right, right_left, right_right, left, right)
    return left, right
  return -1

def global_position(v):
  for child in graph[v].childs:
    graph[child].x += graph[v].x
    global_position(child)
  return

create_graph()  #создаем граф
order_reversal(0, 0)  #нумеруем вершины

left, right = archiving(0) #сжимаем дерево
global_position(0)  #считаем абсолютные координаты

Отрисовка сжатого дерева

In [ ]:
fig = go.Figure()

for node in graph:
  for child in node.childs:
    v0 = node
    v1 = graph[child]
    fig.add_trace(go.Scatter(
        x=[v0.x, v1.x],
        y=[-v0.hight, -v1.hight],
        line=dict(color='green', width=2)
    ))

for node in graph:
  fig.add_trace(go.Scatter(
      x=[node.x],
      y=[-node.hight],
      mode='markers',
      marker=dict(size=[10], color='black'),
  ))

fig.update_layout(  
    autosize=True,
    width=1000,
    height=500,
    margin=dict(l=50, r=50, b=50, t=50, pad=4),
    showlegend = False
)

print("Ширина дерева = ", right.max() - left.min())
fig.show()

Отрисовка несжатого дерева

In [ ]:
fig_2 = go.Figure()

for node in graph:
  for child in node.childs:
    v0 = node
    v1 = graph[child]
    fig_2.add_trace(go.Scatter(
        x=[v0.label, v1.label],
        y=[-v0.hight, -v1.hight],
        line=dict(color='green', width=2)
    ))

for node in graph:
  fig_2.add_trace(go.Scatter(
      x=[node.label],
      y=[-node.hight],
      mode='markers',
      marker=dict(size=[10], color='black'),
  ))

fig_2.update_layout(  
    autosize=True,
    width=1000,
    height=500,
    margin=dict(l=50, r=50, b=50, t=50, pad=4),
    showlegend = False
)
w = max([node.label for node in graph]) - min([node.label for node in graph])
print("Ширина дерева = ", w)
fig_2.show()